## Santa Barbara Weather Forecast Model Evaluation
### UCSB Climate Variation and Changes Research Group
#### Advisor: Professor Charles Jones
#### Author: Pippa Lin

In [1]:
import numpy as np
import pandas as pd
import altair as alt
import math
import os

### Introduction:
This research project aims to interpret present and future climates in Santa Barbara from the Numerical Weather Forecast (NWF) model. The data is collected from https://clivac.eri.ucsb.edu/clivac/wrfreal/index.html. In this project, I performed statistical analysis to compare NWF model data and weather station observation, generating root mean square error, mean bias, and correlation by hour to assess model performance.

### 1.Read the data:

In [4]:
folder_path = '/Users/pippalin/Desktop/Climate Research/xskill-mtic1/' 
file_names = os.listdir(folder_path)

# Initialize an empty list to store the data frames
dfs = []

# Loop through the list of file names and read each file into a Pandas DataFrame
for file_name in file_names:
    # Check if the file is a CSV file
    if file_name.endswith('.txt'):
        # Read the file into a Pandas DataFrame
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path, skiprows = 7,nrows = 73) # since files with > 73 rows has nan values after row 73
        # Change the column name of the data frame
        df.set_axis(['yyyy','mm', 'dd', 'hh', 'min','ss','modtemp','modrh','modwsp','modwd','yyyy1','mm1', 'dd1', 'hh1', 'mm1','ss1','obstemp','obsrh','obswsp','obswd'], axis=1, inplace=True)
        
        # Frist, move the obs-part up by one, covering the first row, and drop the last row:
        df.loc[0:len(df)-2, 'yyyy1':'obswd'] = df.loc[1:len(df)-1, 'yyyy1':'obswd'].values
        df = df.drop(df.index[-1])

        # Add a column hour
        df["hour"] = range(len(df))
        
        dfs.append(df)
        
# Concatenate the data frames into a single data frame
merged_df = pd.concat(dfs).sort_values(by=['yyyy','mm']).reset_index(drop=True)

In [5]:
merged_df

,yyyy,mm,dd,hh,min,ss,modtemp,modrh,modwsp,modwd,...,mm1,dd1,hh1,mm1,ss1,obstemp,obsrh,obswsp,obswd,hour
0,2020,4,25,0,0,0,27.817,44.783,1.64850,4.2949,...,4.0,25.0,0.0,47.0,0.0,30.000,19.0,5.370,11.0,0
1,2020,4,25,1,0,0,30.126,36.512,1.15990,326.0200,...,4.0,25.0,1.0,47.0,0.0,28.890,19.0,12.070,15.0,1
2,2020,4,25,2,0,0,29.504,23.131,6.52000,25.0390,...,4.0,25.0,2.0,47.0,0.0,27.220,21.0,19.220,16.0,2
3,2020,4,25,3,0,0,27.334,26.628,8.96630,15.1050,...,4.0,25.0,3.0,47.0,0.0,26.670,22.0,17.430,18.0,3
4,2020,4,25,4,0,0,26.637,27.462,8.18180,17.1510,...,4.0,25.0,4.0,47.0,0.0,26.670,20.0,18.780,16.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59224,2023,1,17,19,0,0,10.769,52.299,1.02220,242.2200,...,1.0,17.0,19.0,47.0,0.0,10.556,46.0,4.471,340.0,67
59225,2023,1,17,20,0,0,11.540,48.411,0.66567,190.7500,...,1.0,17.0,20.0,47.0,0.0,11.111,42.0,3.575,351.0,68
59226,2023,1,17,21,0,0,12.445,43.218,1.44200,50.4810,...,1.0,17.0,21.0,47.0,0.0,11.667,42.0,2.233,329.0,69
59227,2023,1,17,22,0,0,12.055,43.034,4.59610,12.5020,...,1.0,17.0,22.0,47.0,0.0,11.667,40.0,3.128,41.0,70


#### Check for null values:

In [6]:
merged_df.isnull().sum()

yyyy          0
mm            0
dd            0
hh            0
min           0
ss            0
modtemp       0
modrh         0
modwsp        0
modwd         0
yyyy1         0
mm1           0
dd1           0
hh1           0
mm1           0
ss1           0
obstemp    1025
obsrh      1040
obswsp     1016
obswd      5208
hour          0
dtype: int64

Note that count() function automatically ignores null values:

In [7]:
merged_df.count()

yyyy       59229
mm         59229
dd         59229
hh         59229
min        59229
ss         59229
modtemp    59229
modrh      59229
modwsp     59229
modwd      59229
yyyy1      59229
mm1        59229
dd1        59229
hh1        59229
mm1        59229
ss1        59229
obstemp    58204
obsrh      58189
obswsp     58213
obswd      54021
hour       59229
dtype: int64

### 2.Calculate RMSE, MB, Correlation by hour

- RMSE: $RMSE = \sqrt{\frac{\sum \limits _{i=1} ^{n}(mod-obs)^2}{n}}$
- Mean Bias: $MB = \frac{1}{n} \sum \limits _{i=1} ^{n} (mod - obs)$
- Correlation

#### RMSE:
1. Calcualte $(mod-obs)^2$
2. Group by hour and sum up $(mod-obs)^2$
3. Divide the sum by n and take square root

Although we have null values, the sum() function ignores them

In [8]:
# 1. Calculate the square difference
merged_df["temp_dif"] = (merged_df["obstemp"] - merged_df["modtemp"])**2
merged_df["rh_dif"] = (merged_df["obsrh"] - merged_df["modrh"])**2
merged_df["wsp_dif"] = (merged_df["obswsp"] - merged_df["modwsp"])**2

In [9]:
# 2. Group by hour and sum
sum_temp = merged_df.groupby("hour")["temp_dif"].sum()
sum_rh = merged_df.groupby("hour")["rh_dif"].sum()
sum_wsp = merged_df.groupby("hour")["wsp_dif"].sum()

In [11]:
# 3. Divide the sums by n and take square root, calculate rmse and make it a dataframe
count = merged_df.groupby("hour").count()

# First, write a function to calculate rmse and return rmse
def rmse(data,count):
    rmse = []
    for i in range(len(data)):
        rmse.append(np.sqrt(data[i]/count[i]))
    rmse = pd.DataFrame(rmse,columns = ['rmse'])
    rmse['hour'] = range(0,72)
    return rmse

# Now we calculate rmse for each variable
rmse_temp = rmse(sum_temp,count["obstemp"])
rmse_rh = rmse(sum_rh,count["obsrh"])
rmse_wsp = rmse(sum_wsp,count["obswsp"])

#### Write a funtcion of plotting

In [12]:
def plotrmse(dataframe,plotname):
    # A straight line
    rule = alt.Chart(pd.DataFrame({'Component': [24,48,72]})).mark_rule(color='#D2386C').encode(x='Component')
    
    # Plotting
    plt = alt.Chart(dataframe).mark_line().encode(
    x = 'hour:Q',
    y = 'rmse',
    color=alt.value("#FFAA00")
).properties(
    width = 800,
    height = 300,
    title={
      "text": [plotname],
      "color": "green"
    })
    return plt + rule

In [13]:
plotrmse(rmse_temp,"RMSE plot of Temperature")

alt.LayerChart(...)

In [14]:
plotrmse(rmse_rh,"RMSE plot of Relative Humidity")

alt.LayerChart(...)

In [15]:
plotrmse(rmse_wsp,"RMSE plot of Wind Speed")

alt.LayerChart(...)

<br>

#### MB:
1. Calcualte $(mod-obs)$
2. Group by hour and sum up $(mod-obs)$
3. Divide the sum by n

In [16]:
# 1. Calculate the difference
merged_df["temp_dif"] = merged_df["modtemp"] - merged_df["obstemp"]
merged_df["rh_dif"] = merged_df["modrh"] - merged_df["obsrh"]
merged_df["wsp_dif"] = merged_df["modwsp"] - merged_df["obswsp"]

In [17]:
# 2. Group by hour and sum
sum_temp = merged_df.groupby("hour")["temp_dif"].sum()
sum_rh = merged_df.groupby("hour")["rh_dif"].sum()
sum_wsp = merged_df.groupby("hour")["wsp_dif"].sum()

In [18]:
# 3. Divide the sums by n and take square root, calculate rmse and make it a dataframe
count = merged_df.groupby("hour").count()

# First, write a function to calculate rmse and return rmse
def mb(data,count):
    mb = []
    for i in range(len(data)):
        mb.append(data[i]/count[i])
    mb = pd.DataFrame(mb,columns = ['mb'])
    mb['hour'] = range(0,72)
    return mb

# Now we calculate rmse for each variable
mb_temp = mb(sum_temp,count["obstemp"])
mb_rh = mb(sum_rh,count["obsrh"])
mb_wsp = mb(sum_wsp,count["obswsp"])

#### Write a funtcion of plotting

In [19]:
def plotmb(dataframe,plotname):
    # A straight line
    rule = alt.Chart(pd.DataFrame({'Component': [24,48,72]})).mark_rule(color='#D2386C').encode(x='Component')
    
    # Plotting
    plt = alt.Chart(dataframe).mark_line().encode(
    x = 'hour:Q',
    y = 'mb',
    color=alt.value("#FFAA00")
).properties(
    width = 800,
    height = 300,
    title={
      "text": [plotname],
      "color": "green"
    })
    return plt + rule

In [20]:
plotmb(mb_temp,"MB plot of Temperature")

alt.LayerChart(...)

In [21]:
plotmb(mb_rh,"MB plot of Relative Humidity")

alt.LayerChart(...)

In [22]:
plotmb(mb_wsp,"MB plot of Wind Speed")

alt.LayerChart(...)

<br>

#### Correlation:
1. Create a matrix with:
$$\begin{bmatrix} [mod0h] & [mod1h] & ... & [mod84h] \\ [obs0h] & [obs1h] & ... & [obs84h] \end{bmatrix}$$
where [mod0h] contains a list of all values from 0h
* Note: If the list contains None type value, then the correlation will be None. Therefore, we need to remove the null value in a list and also remove the corresponding obs/mod value. We can write a function to do so:
2. Calculate correlation function between mod_nh and obs_nh

In [23]:
# 1. Define a function which create lists with all values of that hour, also removing null values
def create_list(var):
    return merged_df.groupby("hour")[var].apply(lambda x: x.tolist())

# Calculate the list
modtemp_list = create_list("modtemp")
obstemp_list = create_list("obstemp")
modrh_list = create_list("modrh")
obsrh_list = create_list("obsrh")
modwsp_list = create_list("modwsp")
obswsp_list = create_list("obswsp")

In [24]:
# 2. Since there are NaN for each list, we need to remove nan value and the corresponding index, 
# we first record the index of the missing value and remove i
def index(modlist,obslist):
    # Get the index matrix
    index = [[] for _ in range(len(modlist))]
    for i in range(len(modlist)):
        for j in range(len(modlist[i])):
            if pd.isna(modlist[i][j]) or pd.isna(obslist[i][j]):
                index[i].append(j)
    return index

In [25]:
# 3. Remove the corresponding value from the index
def removeNan(index,modlist,obslist):
    update_mod = [[] for _ in range(len(modlist))]
    update_obs = [[] for _ in range(len(obslist))]
    
    for i in range(len(modlist)):
        for j in range(len(modlist[i])):
            if j not in index[i]:
                update_mod[i].append(modlist[i][j])
                update_obs[i].append(obslist[i][j])
    return update_mod,update_obs

In [26]:
up_modtemp_list,up_obstemp_list = removeNan(index(modtemp_list,obstemp_list),modtemp_list,obstemp_list)
up_modrh_list,up_obsrh_list = removeNan(index(modrh_list,obsrh_list),modrh_list,obsrh_list)
up_modwsp_list,up_obswsp_list = removeNan(index(modwsp_list,obswsp_list),modwsp_list,obswsp_list)

In [27]:
# 4. Create a function to calculate correlation and create a list
def correlation(modlist,obslist):
    correlation = []
    for i in range(len(modlist)):
        correlation.append(np.corrcoef(modlist[i],obslist[i])[0][1])
        
    # Turn into dataframe
    correlation = pd.DataFrame(correlation,columns = ['corr'])
    correlation['hour'] = range(0,72)
    return correlation

In [28]:
corr_temp = correlation(up_modtemp_list,up_obstemp_list)
corr_rh = correlation(up_modrh_list,up_obsrh_list)
corr_wsp = correlation(up_modwsp_list,up_obswsp_list)

#### Write a function for plotting

In [29]:
def plotcorr(dataframe,plotname):
    # A straight line
    rule = alt.Chart(pd.DataFrame({'Component': [24,48,72]})).mark_rule(color='#D2386C').encode(x='Component')
    
    # Plotting
    plt = alt.Chart(dataframe).mark_line().encode(
    x = 'hour:Q',
    y = 'corr',
    color=alt.value("#FFAA00")
).properties(
    width = 800,
    height = 300,
    title={
      "text": [plotname],
      "color": "green"
    })
    return plt + rule

In [30]:
plotcorr(corr_temp,"Correlation plot of Temperature")

alt.LayerChart(...)

In [31]:
plotcorr(corr_rh,"Correlation plot of Relative Humidity")

alt.LayerChart(...)

In [32]:
plotcorr(corr_wsp,"Correlation plot of Wind Speed")

alt.LayerChart(...)

After I took some hour out and calculate manually, the project seems to show the correct statistics (skills)